### Convert embeddings into raw data
Use the following to convert word embedding using's FaceBooks's fastText into raw data for a standard machine learning model. We are assuming a closed world solution with only positive and negative values for our 5 predicates - isChild, isSpouse, isFather, isMother, and isSibling.

In [1]:
import sklearn as sk 
import scipy as sci 
import numpy as np 
import pandas as pd
import pickle

Load in the pickle files containing the word encodings and the entity map

In [86]:
words = pickle.load(open('embeddings/word_vectors.pkl','rb'))
key = pickle.load(open('processed/large_set/entities_map_large.pkl','rb'))

Load in the files containing the positive and negative triplets of (entity,predicate,entity)

In [88]:
converter = dict.fromkeys(range(4), lambda s: s.decode('utf-8'))
pos = np.loadtxt(r'processed/large_set/positiveTriplets_unique_large.txt',delimiter=' ',dtype=str, converters=converter)
neg = np.loadtxt(r'processed/large_set/negativeTriplets_more_large.txt',delimiter=' ',dtype=str, converters=converter)
pos.shape,neg.shape

((37395, 4), (37391, 4))

Make a list of the entities for looping over all of them later

In [89]:
pred_key = {'P22':'isFather','P25':'isMother','P26':'isSpouse','P3373':'isSibling','P40':'isChild'}
pred_key

{'P22': 'isFather',
 'P25': 'isMother',
 'P26': 'isSpouse',
 'P3373': 'isSibling',
 'P40': 'isChild'}

In [90]:
entities = np.unique(pos[:,0])
predicates = np.unique(pos[:,1])
entities.shape,predicates.shape

((3195,), (5,))

A few helper functions for the main data converting tool

In [91]:
def encode_word(word):
    """
        Encodes a word if found in the word 
        encoding dictionary; otherwise returns
        an empty array.
    """
    if word in words.keys():
        return(words[word])
    max_len = max([words[x].shape[0] for x in words])
    return(np.empty(max_len))

def encode_entity(entity):
    """
        Builds an entity encoding by computing
        the average of all the word encodings.
        Ignores any words that don't have an 
        encoding in the words_vectors.txt file.
    """
    all_words = np.array([encode_word(x) for x in key[entity].split(' ') if x in words.keys()])
    if all_words.shape[0]==0:
        return(np.array([np.NaN]*300,dtype=float).transpose())
    return(all_words.mean(axis=0))

### Primary Function
The build_rows function builds the appropriate rows of numerical data for a given entity. This is done by considering all triplets with this entity as the first object. Then it encodes all entities in this subset, including itself, and concatenates each encoding. Finally it looks as the predicate for each entity-entity pair and assigns 1,0, or -1 depending on if the predicate is true, unknown, or false for that entity-entity pair. The result is a small dataframe of all the rows of this form where the first entity is the argument.

In [146]:
def build_rows(entity):
    """
        Builds a dataframe out of the triplets
        containing entity as the first object.
    """
    encoding = encode_entity(entity)
    temp_pos = pos[pos[:,0]==entity,:]
    temp_neg = neg[neg[:,0]==entity,:]
    pairs = []
    rows = []
    bad_encodings = []
    for each_obj in np.concatenate([np.unique(temp_pos[:,2]),np.unique(temp_neg[:,2])]):        
        try:
            obj_encode = encode_entity(each_obj)
            
            if not all(np.isnan(obj_encode)):
                # Get the positive & negative triplet's objects
                temp2_pos = temp_pos[temp_pos[:,2]==each_obj,:]
                temp2_neg = temp_neg[temp_neg[:,2]==each_obj,:]

                # Build outputs from which predicates are in triplets
                pos_preds = np.array([x in np.unique(temp2_pos[:,1]) for x in predicates],dtype=int)
                neg_preds = np.array([x in np.unique(temp2_neg[:,1]) for x in predicates],dtype=int)
                preds = pos_preds - neg_preds

                # Update the data
                row = np.concatenate([encoding,obj_encode,preds])
                rows.append(row)
                pairs.append([entity,each_obj])
                
            else:
                #print('Entity ' + obj_encode + ' has no encoding!')
                bad_encodings.append(each_obj)
        except:
            #print("didn't work for "+ str(entity) + ", " + str(each_obj))  
            bad_encodings.append(each_obj)
 
    rows = np.array(rows)
    pairs = np.array(pairs)
    if rows.shape[0] == 0:
        return({'df':np.NaN,'bad_encodings':np.NaN})
    df = pd.concat([pd.DataFrame(pairs),pd.DataFrame(rows)],ignore_index=True,axis=1)
    col_names = ['EntityA','EntityB']+['a'+str(x) for x in range(300)]+['b'+str(x) for x in range(300)]+['p'+str(x) for x in range(5)]
    df.columns = col_names
    return({'df':df,'bad_encodings':bad_encodings})

entity = "Q107507"
test = build_rows(entity)['df']
test.shape

(27, 607)

Running build_rows on all the entities to generate the main dataset

In [147]:
all_rows = pd.DataFrame([])
bad_encodings = []
for entity in entities:
    rows = build_rows(entity)
    try:
        all_rows = pd.concat([all_rows,rows['df']])
        [bad_encodings.append(x) for x in rows['bad_encodings']]
    except:
        print ('Entity: '+entity+ ' failed!')
        bad_encodings.append(entity)
        
bad_encodings = np.unique(np.array(bad_encodings))

Save the data to a file

In [149]:
all_rows.to_csv('EncodedDataLrg.csv',index=False)

Encode all entities and save as pickle file for future use

In [85]:
Encoded = {entity:encode_entity(entity) for entity in entities}
pickle.dump( Encoded, open( "embeddings/entity_embeddings_lrg.pkl", "wb" ) )
pickle.dump( bad_encodings, open( "embeddings/bad_entities_lrg.pkl", "wb") )

In [101]:
[key[x] for x in all_rows['EntityA'].unique()[0:100]]

['Margaret of Foix',
 'Frederik, Crown Prince of Denmark',
 'George Windsor, Earl of St Andrews',
 'Henry I of England',
 'Margrethe II of Denmark',
 'Henry II of England',
 'Helen of the Palatinate',
 'Robert VII, Lord of BÃ\x83Â©thune',
 'Darius III',
 'Shigeko Higashikuni',
 'Kazuko Takatsukasa',
 'Elizabeth Juliana of Schleswig-Holstein-SÃ\x83Â¸nderburg-Nordborg',
 'Isabel Madalena de Hesse-Darmstadt',
 'Margaret of Valois, Countess of Blois',
 'YelÃ\x83Â¼ Bei',
 'Groucho Marx',
 'Prince Kusakabe',
 'Julius Caesar',
 'Frederick William I of Prussia',
 'Lengvenis',
 'Henry the Lion',
 'Joseph I of Portugal',
 'Sancho I of Portugal',
 'Simeon Saxe-Coburg-Gotha',
 'Queen Elizabeth The Queen Mother',
 'Charles Stuart, 1st Earl of Lennox',
 'Prince Oscar Bernadotte',
 'Jamie Lee Curtis',
 'Sibylle of Baden',
 'Beatrice of Baden',
 'Anna of Brunswick-LÃ\x83Â¼neburg',
 'Joanna of Aragon, Queen of Naples',
 'Christine Marie of France',
 'Ferdinand I of Portugal',
 'Carrie Fisher',
 'Prince

(64950, 607)